In [1]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import make_moons,make_classification,make_regression,make_circles
import sys
from mpl_toolkits.mplot3d import Axes3D 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA,KernelPCA
np.random.seed(23)
np.set_printoptions(precision=3,suppress=True)
plt.style.use('seaborn')

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

def _sym_decorrelation(W):
    """ Symmetric decorrelation
    i.e. W <- (W * W.T) ^{-1/2} * W
    """
    s, u = np.linalg.eigh(np.dot(W, W.T))
    # u (resp. s) contains the eigenvectors (resp. square roots of
    # the eigenvalues) of W * W.T
    return np.dot(np.dot(u * (1./(np.sqrt(s+1e-8))+1e-8), u.T), W)

In [2]:
# import data
mnist = input_data.read_data_sets('../../Dataset/MNIST/', one_hot=True)
train_data,train_label,val_data,val_label,test_data,test_label = mnist.train.images,mnist.train.labels,mnist.validation.images,mnist.validation.labels,mnist.test.images, mnist.test.labels
print(train_data.shape,train_label.shape,train_data.max(),train_data.min())
print(val_data.shape,val_label.shape,val_data.max(),val_data.min())
print(test_data.shape,test_label.shape,test_data.max(),test_data.min())

Extracting ../../Dataset/MNIST/train-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10) 1.0 0.0
(5000, 784) (5000, 10) 1.0 0.0
(10000, 784) (10000, 10) 1.0 0.0


In [ ]:
#  create class
def arctan(x):  return tf.arctan(x)
def tan(x)   :  return tf.tan(x)
def d_arctan(x):return 1/(1+x**2)
def d_tanh(x):  return 1/tf.cos(x)

class FNN():
    
    def __init__(self,input,output,act,d_act):
        self.w = tf.Variable(tf.random.normal())
        self.mw,self.mv = np.zeros_like()
        
        self.a = np.ones(output)
        self.b = np.ones(output)
        self.c = np.zeros(output)
        
        self.act  = act
        self.d_act= d_act

In [53]:
def tf_arctan(x): return tf.atan(x)
def tf_tan(x)   : return tf.tan(x)

def d_tf_arctan(x): return 1/(1+x**2)
def d_tf_tan(x):    return 1/(tf.cos(x)**2)

class PCA_Layer():

    def __init__(self,inc,outc,act=tf_arctan,d_act=d_tf_arctan):
        
        if outc == 1:
            self.w = tf.Variable(self.norm(tf.random_normal([inc,outc],stddev=0.05,seed=2)))
        else:
            self.w = tf.Variable(self.sym_decorrelation(tf.random_normal([inc,outc],stddev=0.05,seed=2)))
        self.a = tf.Variable(tf.ones([outc]))
        self.b = tf.Variable(tf.ones([outc]))
        self.c = tf.Variable(tf.zeros([outc]))
        
        self.mw,self.vw = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.ma,self.va = tf.Variable(tf.zeros_like(self.a)),tf.Variable(tf.zeros_like(self.a))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.mc,self.vc = tf.Variable(tf.zeros_like(self.c)),tf.Variable(tf.zeros_like(self.c))
        
        self.act,self.d_act = act,d_act

    def feedforward_linear(self,input=None):
        self.input  = input
        self.layer  = tf.matmul(self.input,self.w) 
        loss = self.w @ tf.transpose(self.input) @ self.input @ self.w
        return self.layer,loss
    def backprop_linear(self):
        gradw      = -2* tf.transpose(self.input) @ self.input @ self.w
        grad_pass  = -2*self.input@self.w@tf.transpose(self.w)
        
        update_w = []
        update_w.append(tf.assign( self.mw,self.mw*beta1 + (1-beta1) * (gradw)   ))
        update_w.append(tf.assign( self.vw,self.vw*beta2 + (1-beta2) * (gradw ** 2)   ))
        m_hatw = self.mw / (1-beta1)
        v_hatw = self.vw / (1-beta2)
        adam_midw = m_hatw *  learning_rate/(tf.sqrt(v_hatw) + adam_e)
        update_w.append(tf.assign(self.w,self.sym_decorrelation(tf.subtract(self.w,adam_midw ))))
        
        return grad_pass,update_w

    def feedforward_nonlinear(self,input=None):
        self.input  = input
        self.layer  = tf.matmul(self.input,self.w) 
        self.layerA = self.a*self.act(self.b*self.layer) + self.c
        loss = self.w @ tf.transpose(self.input) @ self.input @ self.w
        return self.layerA,loss
    def backprop_nonlinear(self,gradient):
        
        grada = tf.reduce_mean(gradient * self.act(self.b*self.layer),0)
        gradb = tf.reduce_mean(gradient * self.a * self.d_act(self.b*self.layer) * self.layer,0)
        gradc = tf.reduce_mean(gradient,0)
        gradw = tf.transpose(self.input) @ (gradient * self.a * self.d_act(self.b*self.layer) * self.b) - 2 * tf.transpose(self.input) @ self.input @ self.w
        grad_pass = (gradient * self.a * self.d_act(self.b*self.layer) * self.b) @ tf.transpose(self.w) - 2 * self.input@self.w@tf.transpose(self.w)

        update_w = []
        
        update_w.append(tf.assign( self.mw,self.mw*beta1 + (1-beta1) * (gradw)   ))
        update_w.append(tf.assign( self.vw,self.vw*beta2 + (1-beta2) * (gradw ** 2)   ))
        m_hatw    = self.mw / (1-beta1)
        v_hatw    = self.vw / (1-beta2)
        adam_midw = m_hatw *  learning_rate/(tf.sqrt(v_hatw) + adam_e)
        update_w.append(tf.assign(self.w,self.sym_decorrelation(tf.subtract(self.w,adam_midw))))
        
        update_w.append(tf.assign( self.ma,self.ma*beta1 + (1-beta1) * (grada)   ))
        update_w.append(tf.assign( self.va,self.va*beta2 + (1-beta2) * (grada ** 2)   ))
        m_hata    = self.ma / (1-beta1)
        v_hata    = self.va / (1-beta2)
        adam_mida = m_hata *  learning_rate/(tf.sqrt(v_hata) + adam_e)
        update_w.append(tf.assign(self.a,self.stand(tf.subtract(self.a,adam_mida))))
        
        update_w.append(tf.assign( self.mb,self.mb*beta1 + (1-beta1) * (gradb)   ))
        update_w.append(tf.assign( self.vb,self.vb*beta2 + (1-beta2) * (gradb ** 2)   ))
        m_hatb    = self.mb / (1-beta1)
        v_hatb    = self.vb / (1-beta2)
        adam_midb = m_hatb *  learning_rate/(tf.sqrt(v_hatb) + adam_e)
        update_w.append(tf.assign(self.b,self.stand(tf.subtract(self.b,adam_midb))))
        
        update_w.append(tf.assign( self.mc,self.mc*beta1 + (1-beta1) * (gradc)   ))
        update_w.append(tf.assign( self.vc,self.vc*beta2 + (1-beta2) * (gradc ** 2)   ))
        m_hatc    = self.mc / (1-beta1)
        v_hatc    = self.vc / (1-beta2)
        adam_midc = m_hatb *  learning_rate/(tf.sqrt(v_hatc) + adam_e)
        update_w.append(tf.assign(self.c,self.stand(tf.subtract(self.c,adam_midc))))

        return grad_pass,update_w
    
    def stand(self,w):
        mean,variance = tf.nn.moments(w,0)
        return (w-mean)/variance
    def norm(self,W):
        W = W/(tf.sqrt(tf.reduce_sum(W**2)+1e-8))
        return W
    def sym_decorrelation(self,W):
        s, u = tf.linalg.eigh(W@tf.transpose(W))
        result = (u * (1./(tf.sqrt(s+1e-5)+1e-5))) @ tf.transpose(u) @ W
        return result


In [49]:
# set the hyper
learning_rate = 0.000008
beta1,beta2,adam_e = 0.9,0.999,1e-8
mini_batch_size    = 10

In [ ]:
# create layers 
l1 = PCA_Layer(784,128)
l2 = PCA_Layer(128,32)
l3 = PCA_Layer(32,16)
l4 = PCA_Layer(16,2)

x = tf.placeholder(tf.float32,[mini_batch_size,784])

layer1,l1l = l1.feedforward_nonlinear(x)
layer2,l2l = l2.feedforward_nonlinear(layer1)
layer3,l3l = l3.feedforward_nonlinear(layer2)
layer4,l4l = l4.feedforward_linear(layer3)

loss = tf.reduce_mean(l1l) + tf.reduce_mean(l2l) + tf.reduce_mean(l3l) + tf.reduce_mean(l4l)

grad4,grad4_update = l4.backprop_linear()
grad3,grad3_update = l3.backprop_nonlinear(grad4)
grad2,grad2_update = l2.backprop_nonlinear(grad3)
grad1,grad1_update = l1.backprop_nonlinear(grad2)

In [52]:
# start
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
val_data_copy = np.copy(val_data)

for iter in range(1):
    
    val_data_copy  = shuffle(val_data_copy)
    for current_batch_index in range(0,len(val_data_copy),mini_batch_size):
        print(current_batch_index)



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2